## Steam Game Review Analysis

In [1]:
from pyspark.sql import SparkSession, functions, types, Row
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
# pip install matplotlib
# pip install seaborn

In [3]:
spark = SparkSession.builder.appName('Steam Game Data Preprocessing').getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/13 13:04:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/13 13:04:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
game_schema = types.StructType([
    types.StructField('url', types.StringType()),
    types.StructField('types', types.StringType()),
    types.StructField('name', types.StringType()),
    types.StructField('desc_snippet', types.StringType()),
    types.StructField('recent_reviews', types.StringType()),
    types.StructField('all_reviews', types.StringType()),
    types.StructField('release_date', types.DateType()),
    types.StructField('developer', types.StringType()),
    types.StructField('publisher', types.StringType()),
    types.StructField('popular_tags', types.StringType()),
    types.StructField('game_details', types.StringType()),
    types.StructField('languages', types.StringType()),
    types.StructField('achievements', types.IntegerType()),
    types.StructField('genre', types.StringType()),
    types.StructField('game_description', types.StringType()),
    types.StructField('mature_content', types.StringType()),
    types.StructField('minimum_requirements', types.StringType()),
    types.StructField('recommended_requirements', types.StringType()),
    types.StructField('original_price', types.StringType()),
    types.StructField('discount_price', types.StringType()),
])

In [5]:
game_df = spark.read.csv('steam_games.csv', header=True, schema=game_schema)
game_df.show(5)


+--------------------+-----+--------------------+--------------------+--------------------+--------------------+------------+-------------------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------+--------------+
|                 url|types|                name|        desc_snippet|      recent_reviews|         all_reviews|release_date|          developer|           publisher|        popular_tags|        game_details|           languages|achievements|               genre|    game_description|      mature_content|minimum_requirements|recommended_requirements|original_price|discount_price|
+--------------------+-----+--------------------+--------------------+--------------------+--------------------+------------+-------------------+--------------------+--------------------+--------------------+--------------------+-------

In [6]:
# number of missing values in each column 
for col in game_df.columns:
    print(col, "with null values: ", game_df.filter(game_df[col].isNull()).count())

url with null values:  0
types with null values:  54
name with null values:  72
desc_snippet with null values:  164
recent_reviews with null values:  34850
all_reviews with null values:  18393
release_date with null values:  51506
developer with null values:  5762
publisher with null values:  7298
popular_tags with null values:  7444
game_details with null values:  10622
languages with null values:  21081
achievements with null values:  42251
genre with null values:  21774
game_description with null values:  21573
mature_content with null values:  43378
minimum_requirements with null values:  31981
recommended_requirements with null values:  32214
original_price with null values:  25641
discount_price with null values:  37886


In [7]:
# drop duplicated rows in the dataframe

num_rows = game_df.count()

game_df = game_df.dropDuplicates()
non_dup_rows = game_df.count()
print("number of original data rows: ", num_rows)
print("number of data rows after deleting duplicated data: ", non_dup_rows)

Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


CodeCache: size=131072Kb used=34244Kb max_used=34244Kb free=96827Kb
 bounds [0x000000010b1d8000, 0x000000010d388000, 0x00000001131d8000]
 total_blobs=13081 nmethods=12142 adapters=851
 compilation: disabled (not enough contiguous free space left)
number of original data rows:  51898
number of data rows after deleting duplicated data:  51874


In [8]:
# drop unused columns
game_df = game_df.drop('genre')

In [9]:
game_df.printSchema()

root
 |-- url: string (nullable = true)
 |-- types: string (nullable = true)
 |-- name: string (nullable = true)
 |-- desc_snippet: string (nullable = true)
 |-- recent_reviews: string (nullable = true)
 |-- all_reviews: string (nullable = true)
 |-- release_date: date (nullable = true)
 |-- developer: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- popular_tags: string (nullable = true)
 |-- game_details: string (nullable = true)
 |-- languages: string (nullable = true)
 |-- achievements: integer (nullable = true)
 |-- game_description: string (nullable = true)
 |-- mature_content: string (nullable = true)
 |-- minimum_requirements: string (nullable = true)
 |-- recommended_requirements: string (nullable = true)
 |-- original_price: string (nullable = true)
 |-- discount_price: string (nullable = true)



In [10]:
# drop rows with missing url
game_df.select("url", "name") \
    .filter(game_df["url"].contains("\t\t\t\t\t\t\t\t\t")) \
    .show(10)

game_df = game_df.filter(~game_df["url"].contains("\t\t\t\t\t\t\t\t\t")) 

+-------------------+----+
|                url|name|
+-------------------+----+
|\t\t\t\t\t\t\t\t\t"| NaN|
|\t\t\t\t\t\t\t\t\t"| NaN|
|\t\t\t\t\t\t\t\t\t"| NaN|
|\t\t\t\t\t\t\t\t\t"|  22|
|\t\t\t\t\t\t\t\t\t"|  32|
|\t\t\t\t\t\t\t\t\t"| NaN|
|\t\t\t\t\t\t\t\t\t"| NaN|
|\t\t\t\t\t\t\t\t\t"|  24|
|\t\t\t\t\t\t\t\t\t"| NaN|
|\t\t\t\t\t\t\t\t\t"|  54|
+-------------------+----+
only showing top 10 rows



In [11]:
# make game_id column by extracting the id from the url
game_df = game_df.withColumn('game_id', functions.element_at(functions.split(game_df['url'], '/'), -3)).cache()
game_df.select(['game_id']).show(5)

+-------+
|game_id|
+-------+
|   6998|
| 674020|
| 389430|
| 414870|
| 246620|
+-------+
only showing top 5 rows



In [12]:
# remove rows with bundle type
game_df = game_df.filter(game_df['types'] != "bundle")

In [13]:
# remove rows with non-integer game id
game_df = game_df.filter(game_df['game_id'].cast("int").isNotNull())

In [14]:
# show duplicated records based on game_id
game_df.groupby('game_id').count().where('count > 1').count()


0

In [15]:
# split developers into an array
game_df = game_df.withColumn("developers", functions.split(game_df['developer'], ",")) \
    .drop("developer")

In [16]:
# split "all_reviews" column into 3 columns
pattern = "^(\w+[\s\w]*),\((\d+[,\d]*)\),- (\d+)%"
game_df = game_df.withColumn("overall_review", functions.regexp_extract(game_df["all_reviews"], pattern, 1)) \
    .withColumn("num_reviews", functions.regexp_extract(game_df["all_reviews"], pattern, 2)) \
    .withColumn("positive_review_percent", functions.regexp_extract(game_df["all_reviews"], pattern, 3))
game_df = game_df.withColumn("num_reviews", functions.regexp_replace(game_df["num_reviews"], ',', ''))
game_df = game_df.withColumn("num_reviews", game_df["num_reviews"].cast("int"))

In [17]:
game_df = game_df.withColumn("overall_review", \
    functions.when(game_df["all_reviews"].rlike("^(\d)"), "<10 reviews") \
    .otherwise(game_df["overall_review"]))

In [18]:
# create a dataframe excluding rows with missing tag
nonnull_tag_df = game_df.where(~game_df['popular_tags'].isNull()) \
    .where(~game_df['popular_tags'].contains("NaN")) \
    .where(game_df['popular_tags'].cast("int").isNull())

In [19]:
# filter out popular_tags containing dates
expr = "[a-zA-Z]\s+[0-9]{1,2},\s+[0-9]{4}"
removed_dates = nonnull_tag_df.where(~nonnull_tag_df["popular_tags"].rlike(expr))

In [20]:
# split "popular tags" column into array and name the column as "tag"
game_df = removed_dates.withColumn("tags", functions.split(removed_dates['popular_tags'], ","))

In [21]:
# drop types since we only have one type
game_df = game_df.drop(game_df["types"])

In [22]:
# replace all values in column "original_price" containing the word "Free" with "Free"
game_df = game_df.withColumn("original_price", \
    functions.when(game_df["original_price"].rlike("(Free)"), "$0.0") \
    .otherwise(game_df["original_price"]))
game_df = game_df.withColumn("original_price", functions.regexp_extract(game_df["original_price"], "^\$(\d+.\d+)", 1))
game_df = game_df.withColumn("original_price", game_df["original_price"].cast("double"))

In [23]:
game_df.select("discount_price").where(game_df["discount_price"].contains("0")).show(10)

+--------------------+
|      discount_price|
+--------------------+
|              $16.80|
|              $20.68|
|Recommended:,Stor...|
|              $27.50|
|              $43.40|
|Minimum:,OS:,Win ...|
|Recommended:,OS:,...|
|             $304.89|
|              $40.72|
|               $0.99|
+--------------------+
only showing top 10 rows



In [24]:
game_df = game_df.withColumn("positive_review_percent", game_df["positive_review_percent"].cast("double"))

In [25]:
game_df.printSchema()

root
 |-- url: string (nullable = true)
 |-- name: string (nullable = true)
 |-- desc_snippet: string (nullable = true)
 |-- recent_reviews: string (nullable = true)
 |-- all_reviews: string (nullable = true)
 |-- release_date: date (nullable = true)
 |-- publisher: string (nullable = true)
 |-- popular_tags: string (nullable = true)
 |-- game_details: string (nullable = true)
 |-- languages: string (nullable = true)
 |-- achievements: integer (nullable = true)
 |-- game_description: string (nullable = true)
 |-- mature_content: string (nullable = true)
 |-- minimum_requirements: string (nullable = true)
 |-- recommended_requirements: string (nullable = true)
 |-- original_price: double (nullable = true)
 |-- discount_price: string (nullable = true)
 |-- game_id: string (nullable = true)
 |-- developers: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- overall_review: string (nullable = true)
 |-- num_reviews: integer (nullable = true)
 |-- positive_review_

In [26]:
# write the final df to parquet file
game_df.write.parquet('cleaned_steam_games', mode='overwrite')

23/11/13 13:04:49 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/11/13 13:04:49 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/11/13 13:04:49 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/11/13 13:04:50 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/11/13 13:04:50 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/11/13 13:04:50 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


23/11/14 11:25:40 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 957752 ms exceeds timeout 120000 ms
23/11/14 11:25:40 WARN SparkContext: Killing executors is not supported by current scheduler.
23/11/14 11:42:05 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:641)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1111)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:244)
	at sc